In [5]:
import numpy as np
import pandas as pd
import seaborn as sns
import string
import os
import requests
import tensorflow_hub as hub
import time
import tensorflow
import matplotlib as plt

from requests import get
from bs4 import BeautifulSoup
from warnings import warn
from time import sleep
from random import randint
from xgboost import XGBClassifier
from tqdm import tqdm
from time import sleep
from matplotlib.figure import Figure
from PIL import Image

from pathlib import Path
from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.model_selection import train_test_split, GridSearchCV
from nltk.stem.porter import *
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression, SGDClassifier
from sklearn.metrics import accuracy_score, hamming_loss
from sklearn.multiclass import OneVsRestClassifier
from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC

from tensorflow.keras import layers, Sequential, callbacks, optimizers, utils, models, applications
from tensorflow.keras.metrics import CategoricalAccuracy
from tensorflow.keras.applications.inception_v3 import InceptionV3
from tensorflow.keras.regularizers import l2, l1
from tensorflow.keras.preprocessing import image
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.preprocessing.text import Tokenizer

2023-08-12 14:14:00.185038: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-08-12 14:14:00.346268: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-08-12 14:14:00.348911: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-08-12 14:14:01.874030: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
open('/content/gdrive/My Drive/')

FileNotFoundError: ignored

In [ ]:
!pip install PIL

ERROR: Could not find a version that satisfies the requirement PIL (from versions: none)
ERROR: No matching distribution found for PIL


In [33]:
folder_path = '/home/jessica/code/chooj202/movie_genre_prediction/notebooks/'
dataframes = []
for file_name in os.listdir(folder_path):
    if file_name.endswith('long_sypnosis.csv'):
        file_path = os.path.join(folder_path, file_name)
        df = pd.read_csv(file_path)
        dataframes.append(df)
long_df = pd.concat(dataframes, ignore_index=True)

In [34]:
# merged_df = pd.read_csv('/content/drive/MyDrive/merged_500.csv')

In [37]:
long_df = long_df.drop(columns = ["Unnamed: 0", "movie", "image_url"])

In [21]:
merged_df = merged_df.drop(columns = ["Unnamed: 0", "index", "movie", "plot", "image_url"])

In [22]:
corrupted_list = []

for img in tqdm(range(merged_df.shape[0])):
    try:
        images_path = f"/home/jessica/code/chooj202/movie_genre_prediction/notebooks/merged/{merged_df['imdb_id'][img]}.jpg"
        image = Image.open(images_path)
    except:
        corrupted_list.append(merged_df['imdb_id'][img])

100%|█████████████████████████████████████████████████████████████████████████████████| 4237/4237 [00:00<00:00, 9149.23it/s]


In [23]:
merged_df = merged_df.loc[~merged_df['imdb_id'].isin(corrupted_list)]

In [24]:
merged_df

,imdb_id,genre
0,tt0111161,['Drama']
1,tt0068646,"['Crime', ' Drama']"
2,tt0259534,"['Animation', ' Action', ' Adventure']"
3,tt0468569,"['Action', ' Crime', ' Drama']"
4,tt0108052,"['Biography', ' Drama', ' History']"
...,...,...
4232,tt0076752,"['Action', ' Adventure', ' Thriller']"
4233,tt0366551,"['Adventure', ' Comedy']"
4234,tt3721936,"['Adventure', ' Drama', ' Romance']"
4235,tt13539646,"['Action', ' Adventure', ' Drama']"


In [14]:
merged_df = merged_df.drop(columns = ['imdb_id', 'Unnamed: 0', 'index', 'image_url'])

In [38]:
long_df = long_df.astype(str).drop_duplicates()

In [40]:
long_df.sort_values('imdb_id')

,imdb_id,genre,sypnosis
3014,tt0004972,"['Drama', ' History', ' War']",The Stoneman family finds its friendship with ...
2906,tt0006864,"['Drama', ' History']",The story of a poor young woman separated by p...
4045,tt0009968,"['Drama', ' Romance']","A frail waif, abused by her brutal boxer fathe..."
1640,tt0012349,"['Comedy', ' Drama', ' Family']","The Tramp cares for an abandoned child, but ev..."
2690,tt0013442,"['Fantasy', ' Horror']",Vampire Count Orlok expresses interest in a ne...
...,...,...,...
1613,tt9779516,"['Biography', ' Drama', ' Music']",The true-life story of Christian music star Je...
1272,tt9784798,"['Biography', ' Drama', ' History']","Offered a plea deal by the FBI, William O'Neal..."
1055,tt9848626,"['Animation', ' Adventure', ' Comedy']","After one experiment, Johnny turns into a mons..."
3861,tt9873892,"['Action', ' Comedy', ' Mystery']",A series of eerie events thrusts an unlikely t...


In [41]:
long_df["genre"] = (
    long_df["genre"]
    .apply(eval)
    .apply(lambda x: [genre.strip() for genre in x])
    )

In [28]:
merged_df = merged_df.dropna()

In [29]:
merged_df.head()

,imdb_id,genre
0,tt0111161,[Drama]
1,tt0068646,"[Crime, Drama]"
2,tt0259534,"[Animation, Action, Adventure]"
3,tt0468569,"[Action, Crime, Drama]"
4,tt0108052,"[Biography, Drama, History]"


In [42]:
multilabel_binarizer = MultiLabelBinarizer()
multilabel_binarizer.fit(long_df['genre'])

#transform target variable
y = multilabel_binarizer.transform(long_df['genre'])
genre_names = multilabel_binarizer.classes_

# Adding
for i in range(len(genre_names)):
    long_df[f"{genre_names[i]}"] = y[:,i]

In [45]:
long_df

,imdb_id,genre,sypnosis,Action,Adventure,Animation,Biography,Comedy,Crime,Drama,...,Horror,Music,Musical,Mystery,Romance,Sci-Fi,Sport,Thriller,War,Western
0,tt0259534,"[Animation, Action, Adventure]",An anime adaptation of the Hindu epic the Rama...,1,1,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,tt0468569,"[Action, Crime, Drama]",When the menace known as the Joker wreaks havo...,1,0,0,0,0,1,1,...,0,0,0,0,0,0,0,0,0,0
2,tt0167260,"[Action, Adventure, Drama]",Gandalf and Aragorn lead the World of Men agai...,1,1,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
3,tt9362722,"[Animation, Action, Adventure]","Miles Morales catapults across the Multiverse,...",1,1,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,tt0075143,"[Action, Adventure, Biography]","The story of prophet ""Muhammad"" and the delive...",1,1,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4619,tt0109198,"[Action, Adventure, Drama]",Four prostitutes join together to travel the O...,1,1,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
4620,tt0276830,"[Adventure, Fantasy, Western]","In 1870, after a brutal run-in with an outlaw ...",0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
4621,tt0120891,"[Action, Comedy, Sci-Fi]",The two best special agents in the Wild West m...,1,0,0,0,1,0,0,...,0,0,0,0,0,1,0,0,0,0
4622,tt2479478,"[Action, Adventure, Comedy]",An outlaw who was raised by Native Americans d...,1,1,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0


(3943, 21)

In [ ]:
X.shape

(3943, 256, 256, 3)

In [ ]:
np.save('y_array', y)

In [ ]:
merged_df.to_csv("jess_small_dataset.csv")

In [ ]:
# # resize all images to 256 x 256
# # convert images to arrays (ignore corrupted images)
# width, height = 256, 256
# image_array = []

# for i in tqdm(range(merged_df.shape[0])):
#     try:
#         image_path = f"/home/jessica/code/chooj202/movie_genre_prediction/raw_data/posters/{merged_df['imdb_id'][i]}.jpg"
#         img = Image.open(image_path)
#         input_arr = image.img_to_array(img)
#         input_arr = input_arr/255.0
#         image_array.append([merged_df['imdb_id'][i], input_arr])
#         time.sleep(1)
#     except:
#         pass

# img_array_df = pd.DataFrame(image_array, columns=["imdb_id", "image_array"])

100%|█████████████████████████████████████████████████████████████████████████████| 1702/1702 [00:00<00:00, 7787.25it/s]


In [ ]:
imgs = []
images_path = f"/home/jessica/code/chooj202/movie_genre_prediction/raw_data/posters/{merged_df['imdb_id'][i]}.jpg"
for img in tqdm(range(merged_df.shape[0])):
    image = Image.open(images_path)
    image = image.resize((256, 256))
    imgs.append(np.array(image))
    time.sleep(1)

100%|███████████████████████████████████████████████████████████████████████████████| 1398/1398 [24:41<00:00,  1.06s/it]


In [ ]:
X = np.array(imgs)

np.save('X_array', X)

In [ ]:
new_df = pd.merge(img_array_df, merged_df, how = 'right', on = 'imdb_id')

In [ ]:
new_df

,imdb_id,image_array,plot,Action,Adventure,Animation,Biography,Comedy,Crime,Drama,...,Horror,Music,Musical,Mystery,Romance,Sci-Fi,Sport,Thriller,War,Western
0,tt0075143,[[[0.99607843 0.99607843 1. ]\n [0.992...,"The story of prophet ""Muhammad"" and the delive...",1,1,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,tt0120815,[[[0.94509804 0.9254902 0.9490196 ]\n [0.819...,Opening with the Allied invasion of Normandy o...,0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,1,0
5,tt0118799,[[[0.98039216 0.95686275 0.8627451 ]\n [0.980...,"In 1930s Italy, a carefree Jewish waiter named...",0,0,0,0,1,0,1,...,0,0,0,0,1,0,0,0,0,0
8,tt0116860,[[[0.9647059 0.96862745 0.9843137 ]\n [0.988...,"At the Belgrade army hospital, casualties of B...",0,0,0,0,1,0,1,...,0,0,0,0,0,0,0,0,0,0
10,tt0253474,[[[0.5254902 0.5254902 0.5254902 ]\n [0.325...,"In this adaptation of the autobiography ""The P...",0,0,0,1,0,0,1,...,0,1,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2391,tt1073241,[[[0.23921569 0.21960784 0.23529412]\n [0.415...,Thinking Pulitzer Prize and hoping to bring do...,0,0,0,0,0,1,1,...,0,0,0,1,0,0,0,0,0,0
2392,tt0034398,[[[0.11764706 0.11764706 0.11764706]\n [0.105...,"Upon the death of his brother, Larry Talbot re...",0,0,0,0,0,0,0,...,1,0,0,1,1,0,0,0,0,0
2393,tt0478134,[[[0.20784314 0.21176471 0.23137255]\n [0.117...,A retired military investigator works with a p...,0,0,0,0,0,1,1,...,0,0,0,1,0,0,0,0,0,0
2394,tt1787988,[[[0.8352941 0.40784314 0.00392157]\n [0.831...,Seeing reflections of Mumbai under the red lig...,0,0,0,0,0,1,1,...,0,0,0,1,0,0,0,0,0,0


In [46]:
def cleaning(sentence):

    # Basic cleaning
    sentence = sentence.strip() ## remove whitespaces
    sentence = sentence.lower() ## lowercase
    sentence = ''.join(char for char in sentence if not char.isdigit()) ## remove numbers

    # Advanced cleaning
    for punctuation in string.punctuation:
        sentence = sentence.replace(punctuation, '') ## remove punctuation

    tokenized_sentence = word_tokenize(sentence) ## tokenize
    stop_words = set(stopwords.words('english')) ## define stopwords

    tokenized_sentence_cleaned = [ ## remove stopwords
    w for w in tokenized_sentence if not w in stop_words]

    lemmatized = [
    WordNetLemmatizer().lemmatize(word, pos = "v")
        for word in tokenized_sentence_cleaned
    ]

    cleaned_sentence = ' '.join(word for word in lemmatized)

    return cleaned_sentence

In [48]:
long_df['sypnosis'] = long_df['sypnosis'].apply(cleaning)

In [57]:
from skmultilearn.model_selection import iterative_train_test_split

mlb = MultiLabelBinarizer()

def balanced_split(df, mlb, test_size=0.5):
    ind = np.expand_dims(np.arange(len(df)), axis=1)
    mlb.fit_transform(df["genre"])
    labels = mlb.transform(df["genre"])
    ind_train, _, ind_test, _ = iterative_train_test_split(
        ind, labels, test_size
    )
    return df.iloc[ind_train[:, 0]], df.iloc[ind_test[:, 0]]

In [59]:
df_train, df_test = balanced_split(long_df, mlb, test_size=0.2)
# df_val, df_test = balanced_split(df_tmp, test_size=0.5)

In [60]:
df_test

,imdb_id,genre,sypnosis,Action,Adventure,Animation,Biography,Comedy,Crime,Drama,...,Horror,Music,Musical,Mystery,Romance,Sci-Fi,Sport,Thriller,War,Western
2,tt0167260,"[Action, Adventure, Drama]",gandalf aragorn lead world men saurons army dr...,1,1,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
4,tt0075143,"[Action, Adventure, Biography]",story prophet muhammad delivery message god allah,1,1,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
6,tt0120737,"[Action, Adventure, Drama]",meek hobbit shire eight companion set journey ...,1,1,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
8,tt6019206,"[Action, Crime, Thriller]",bride must kill exboss lover bill betray wed r...,1,0,0,0,0,1,0,...,0,0,0,0,0,0,0,1,0,0
10,tt0080684,"[Action, Adventure, Fantasy]",rebel overpower empire luke skywalker begin je...,1,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4534,tt0067411,"[Drama, Western]",gambler prostitute become business partner rem...,0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,1
4536,tt0046303,"[Drama, Western]",weary gunfighter wyoming begin envision quiete...,0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,1
4547,tt0068154,"[Comedy, Western]",bambino try teach brother trinity become outla...,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,1
4599,tt3608930,"[Action, Western]",mysterious stranger random act violence drag t...,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1


In [62]:
X_train = df_train['sypnosis']

In [61]:
X_test = df_test['sypnosis']

In [ ]:
X = np.array(imgs)

In [10]:
X.shape

(3889, 256, 256, 3)

In [ ]:
merged_df = merged_df.drop(columns = ['imdb_id', 'plot'])

In [5]:
y.shape

(3889, 20)

In [ ]:
y = merged_df
y.to_csv("Y.csv")

In [40]:
y = np.load('/content/drive/MyDrive/y500_array.npy')

In [ ]:
y = y.drop(columns = ['Unnamed: 0'])

In [41]:
X = np.load('/content/drive/MyDrive/X500_array.npy')

In [49]:
# X = np.load('X_array.npy')

In [42]:
X_train, X_test, y_train ,y_test = train_test_split(X, y, test_size=0.2)

In [43]:
X_val, X_test, y_val, y_test = train_test_split(X_test, y_test, test_size = 0.5)

In [16]:
X_train.shape

(3111, 256, 256, 3)

In [14]:
X_val.shape

(389, 256, 256, 3)

In [ ]:
X_test.shape

(395, 256, 256, 3)

In [24]:
y_test.shape

(389, 20)

In [16]:
y_val.shape

(389, 20)

In [17]:
y_train.shape

(3111, 20)

## CNN

In [ ]:
from tensorflow.keras import Sequential, layers
from tensorflow.keras import optimizers
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.layers.experimental.preprocessing import Rescaling

def load_own_model():

    model = Sequential()
    model.add(Rescaling(1./255, input_shape=(256,256,3)))

    model.add(layers.Conv2D(16, kernel_size=10, activation='relu'))
    model.add(layers.MaxPooling2D(3))

    model.add(layers.Conv2D(32, kernel_size=8, activation="relu"))
    model.add(layers.MaxPooling2D(3))

    model.add(layers.Conv2D(32, kernel_size=6, activation="relu"))
    model.add(layers.MaxPooling2D(3))

    model.add(layers.Flatten())
    model.add(layers.Dense(100, activation='relu'))
    model.add(layers.Dense(21, activation='softmax'))

    opt = optimizers.Adam(learning_rate=1e-4)
    model.compile(loss='categorical_crossentropy',
                  optimizer=opt,
                  metrics=['accuracy'])

    return model

In [ ]:
model_homemade = load_own_model()
model_homemade.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 rescaling (Rescaling)       (None, 256, 256, 3)       0         
                                                                 
 conv2d (Conv2D)             (None, 247, 247, 16)      4816      
                                                                 
 max_pooling2d (MaxPooling2D  (None, 82, 82, 16)       0         
 )                                                               
                                                                 
 conv2d_1 (Conv2D)           (None, 75, 75, 32)        32800     
                                                                 
 max_pooling2d_1 (MaxPooling  (None, 25, 25, 32)       0         
 2D)                                                             
                                                                 
 conv2d_2 (Conv2D)           (None, 20, 20, 32)        3

In [ ]:
es = EarlyStopping(monitor = 'val_accuracy',
                   mode = 'max',
                   patience = 5,
                   verbose = 1,
                   restore_best_weights = True)

history = model_homemade.fit(X_train, y_train,
                             validation_data = (X_val, y_val),
                             batch_size = 16,
                             epochs = 100,
                             callbacks=[es])

Epoch 1/100
198/198 [==============================] - 18s 23ms/step - loss: 19550.4531 - accuracy: 0.1620 - val_loss: 132950.1562 - val_accuracy: 0.1726
Epoch 2/100
198/198 [==============================] - 4s 20ms/step - loss: 1279690.5000 - accuracy: 0.1677 - val_loss: 3859834.5000 - val_accuracy: 0.1980
Epoch 3/100
198/198 [==============================] - 4s 21ms/step - loss: 9163273.0000 - accuracy: 0.1538 - val_loss: 14607878.0000 - val_accuracy: 0.1853
Epoch 4/100
198/198 [==============================] - 4s 20ms/step - loss: 30560906.0000 - accuracy: 0.1522 - val_loss: 48564364.0000 - val_accuracy: 0.2030
Epoch 5/100
198/198 [==============================] - 4s 20ms/step - loss: 74421184.0000 - accuracy: 0.1506 - val_loss: 105663712.0000 - val_accuracy: 0.1980
Epoch 6/100
198/198 [==============================] - 4s 20ms/step - loss: 139518912.0000 - accuracy: 0.1585 - val_loss: 189814000.0000 - val_accuracy: 0.1472
Epoch 7/100
198/198 [==============================] - 4

In [ ]:
res = model_homemade.evaluate(X_test, y_test)

13/13 [==============================] - 1s 32ms/step - loss: 420876256.0000 - accuracy: 0.2051


In [ ]:
test_accuracy = res[-1]
print(f"test_accuracy = {round(test_accuracy,2)*100} %")

test_accuracy = 21.0 %


## RESNET 50

In [ ]:
!pip3 install nvidia-tensorrt

  Preparing metadata (setup.py) ... done
  Created wheel for tensorrt: filename=tensorrt-8.6.1-py2.py3-none-any.whl size=16972 sha256=0763287a6dc9951ce1b79810c204573546288c31f966df08ce3b978153c65d4a
  Stored in directory: /home/jessica/.cache/pip/wheels/6d/29/56/abdffd4c604f255b5254bef3f1c598ab7811ea020540599438
Successfully built tensorrt


In [ ]:
from tensorflow.keras.applications.resnet50 import ResNet50

def load_model():

    # $CHALLENGIFY_BEGIN

    model = ResNet50(weights="imagenet", include_top=False, input_shape=(256, 256, 3))

    # $CHALLENGIFY_END

    return model

model = load_model()

94765736/94765736 [==============================] - 3s 0us/step


In [ ]:
def set_nontrainable_layers(model):

    # $CHALLENGIFY_BEGIN
    # Set the first layers to be untrainable
    model.trainable = False

    # $CHALLENGIFY_END

    return model

model = set_nontrainable_layers(model)
model.summary()

Model: "resnet50"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 256, 256, 3  0           []                               
                                )]                                                                
                                                                                                  
 conv1_pad (ZeroPadding2D)      (None, 262, 262, 3)  0           ['input_1[0][0]']                
                                                                                                  
 conv1_conv (Conv2D)            (None, 128, 128, 64  9472        ['conv1_pad[0][0]']              
                                )                                                                 
                                                                                           

In [ ]:
from tensorflow.keras import layers, models

def add_last_layers(model):
    '''Take a pre-trained model, set its parameters as non-trainable, and add additional trainable layers on top'''
    # $CHALLENGIFY_BEGIN
    base_model = load_model()
    base_model = set_nontrainable_layers(base_model)
    flatten_layer = layers.Flatten()
    dense_layer = layers.Dense(500, activation='relu')
    prediction_layer = layers.Dense(21, activation='softmax')

    model = models.Sequential([
        base_model,
        flatten_layer,
        dense_layer,
        prediction_layer
    ])
    # $CHALLENGIFY_END
    return model

model = add_last_layers(model)

In [ ]:
from tensorflow.keras import optimizers

def build_model():
    # $CHALLENGIFY_BEGIN
    model = load_model()
    model = add_last_layers(model)

    opt = optimizers.Adam(learning_rate=1e-4)
    model.compile(loss='categorical_crossentropy',
                  optimizer=opt,
                  metrics=['accuracy'])
    return model
    # $CHALLENGIFY_END

model = build_model()

In [ ]:
from tensorflow.keras.applications.resnet50 import preprocess_input

X_train = preprocess_input(X_train)
X_val = preprocess_input(X_val)
X_test = preprocess_input(X_test)

In [ ]:
from tensorflow.keras.callbacks import EarlyStopping

model = build_model()

es = EarlyStopping(monitor = 'val_accuracy',
                   mode = 'max',
                   patience = 5,
                   verbose = 1,
                   restore_best_weights = True)

history = model.fit(X_train, y_train,
                    validation_data=(X_val, y_val),
                    epochs=50,
                    batch_size=16,
                    callbacks=[es])

Epoch 1/50
198/198 [==============================] - 27s 104ms/step - loss: 474.6301 - accuracy: 0.2958 - val_loss: 919.1820 - val_accuracy: 0.3376
Epoch 2/50
198/198 [==============================] - 17s 85ms/step - loss: 1488.0557 - accuracy: 0.3542 - val_loss: 1768.7870 - val_accuracy: 0.2741
Epoch 3/50
198/198 [==============================] - 17s 85ms/step - loss: 2773.7695 - accuracy: 0.3494 - val_loss: 3480.1306 - val_accuracy: 0.2893
Epoch 4/50
198/198 [==============================] - 17s 84ms/step - loss: 4915.2197 - accuracy: 0.3345 - val_loss: 5581.1030 - val_accuracy: 0.2005
Epoch 5/50
198/198 [==============================] - 18s 89ms/step - loss: 7514.8691 - accuracy: 0.3491 - val_loss: 7841.3838 - val_accuracy: 0.1523
Epoch 6/50
198/198 [==============================] - 17s 86ms/step - loss: 9680.1855 - accuracy: 0.2980 - val_loss: 9694.1094 - val_accuracy: 0.2538
Epoch 6: early stopping


In [ ]:
res_resnet = model.evaluate(X_test, y_test)

test_accuracy_resnet = res_resnet[-1]


print(f"test_accuracy_resnet = {round(test_accuracy_resnet,2)*100} %")


13/13 [==============================] - 4s 175ms/step - loss: 871.4994 - accuracy: 0.3190
test_accuracy_resnet = 32.0 %


## VGG 19

In [ ]:
from tensorflow.keras.applications.vgg19 import VGG19

def load_model():

    # $CHALLENGIFY_BEGIN

    model = VGG19(weights="imagenet", include_top=False, input_shape=(256, 256, 3))

    # $CHALLENGIFY_END

    return model

model = load_model()

In [ ]:
def set_nontrainable_layers(model):

    # $CHALLENGIFY_BEGIN
    # Set the first layers to be untrainable
    model.trainable = False

    # $CHALLENGIFY_END

    return model

model = set_nontrainable_layers(model)

In [ ]:
from tensorflow.keras import layers, models

def add_last_layers(model):
    '''Take a pre-trained model, set its parameters as non-trainable, and add additional trainable layers on top'''
    # $CHALLENGIFY_BEGIN
    base_model = load_model()
    base_model = set_nontrainable_layers(base_model)
    flatten_layer = layers.Flatten()
    dense_layer = layers.Dense(500, activation='relu')
    prediction_layer = layers.Dense(21, activation='softmax')

    model = models.Sequential([
        base_model,
        flatten_layer,
        dense_layer,
        prediction_layer
    ])
    # $CHALLENGIFY_END
    return model

model = add_last_layers(model)

In [ ]:
from tensorflow.keras import optimizers

def build_model():
    # $CHALLENGIFY_BEGIN
    model = load_model()
    model = add_last_layers(model)

    opt = optimizers.Adam(learning_rate=1e-4)
    model.compile(loss='categorical_crossentropy',
                  optimizer=opt,
                  metrics=['accuracy'])
    return model
    # $CHALLENGIFY_END

model = build_model()

In [ ]:
from tensorflow.keras.applications.vgg19 import preprocess_input

X_train = preprocess_input(X_train)
X_val = preprocess_input(X_val)
X_test = preprocess_input(X_test)

In [ ]:
from tensorflow.keras.callbacks import EarlyStopping

model = build_model()

es = EarlyStopping(monitor = 'val_accuracy',
                   mode = 'max',
                   patience = 5,
                   verbose = 1,
                   restore_best_weights = True)

history = model.fit(X_train, y_train,
                    validation_data=(X_val, y_val),
                    epochs=50,
                    batch_size=16,
                    callbacks=[es])

Epoch 1/50
198/198 [==============================] - 40s 140ms/step - loss: 445.4320 - accuracy: 0.2685 - val_loss: 814.5861 - val_accuracy: 0.3147
Epoch 2/50
198/198 [==============================] - 25s 124ms/step - loss: 1376.3737 - accuracy: 0.3738 - val_loss: 1995.0945 - val_accuracy: 0.3147
Epoch 3/50
198/198 [==============================] - 24s 120ms/step - loss: 2760.8323 - accuracy: 0.3773 - val_loss: 3863.5579 - val_accuracy: 0.3046
Epoch 4/50
198/198 [==============================] - 23s 118ms/step - loss: 5283.5762 - accuracy: 0.3672 - val_loss: 6335.3896 - val_accuracy: 0.2716
Epoch 5/50
198/198 [==============================] - 26s 133ms/step - loss: 7848.6250 - accuracy: 0.3729 - val_loss: 8933.8457 - val_accuracy: 0.1929
Epoch 6/50
198/198 [==============================] - 27s 134ms/step - loss: 11167.3418 - accuracy: 0.3348 - val_loss: 11987.1484 - val_accuracy: 0.1726
Epoch 6: early stopping


In [ ]:
res_vgg19 = model.evaluate(X_test, y_test)

test_accuracy_vgg19 = res_vgg19[-1]


print(f"test_accuracy_vgg19 = {round(test_accuracy_vgg19,2)*100} %")

13/13 [==============================] - 11s 413ms/step - loss: 849.5046 - accuracy: 0.3139
test_accuracy_vgg19 = 31.0 %


## VGG 16

In [60]:
def alpha_evaluation_score(y_true, y_pred):
    alpha = 1
    beta = 0.25
    gamma = 1

    # compute true positives across training examples and labels
    tp = np.sum(np.logical_and(y_true, y_pred))

    # compute false negatives (Missed Labels) across training examples and labels
    fn = np.sum(np.logical_and(y_true, np.logical_not(y_pred)))

    # compute False Positive across training examples and labels.
    fp = np.sum(np.logical_and(np.logical_not(y_true), y_pred))

    # Compute alpha evaluation score
    alpha_score = (1 - ((beta * fn + gamma * fp ) / (tp +fn + fp + 0.00001)))**alpha

    return alpha_score

In [19]:
def hamming_loss(y_true, y_pred):
    """
	XOR TT for reference -

	A  B   Output

	0  0    0
	0  1    1
	1  0    1
	1  1    0
	"""
    hl_num = np.sum(np.logical_xor(y_true, y_pred))
    hl_den = np.prod(y_true.shape)

    return hl_num/hl_den

In [33]:
def Custom_Hamming_Loss(y_true, y_pred):
  return K.mean(y_true*(1-y_pred)+(1-y_true)*y_pred)

In [48]:
from tensorflow.keras.applications.vgg16 import VGG16

def load_model():

    # $CHALLENGIFY_BEGIN

    model = VGG16(weights="imagenet", include_top=False, input_shape=(256, 256, 3))

    # $CHALLENGIFY_END

    return model

model = load_model()

In [49]:
def set_nontrainable_layers(model):

    # $CHALLENGIFY_BEGIN
    # Set the first layers to be untrainable
    model.trainable = False

    # $CHALLENGIFY_END

    return model

model = set_nontrainable_layers(model)

In [54]:
from tensorflow.keras import layers, models

def add_last_layers(model):
    '''Take a pre-trained model, set its parameters as non-trainable, and add additional trainable layers on top'''
    # $CHALLENGIFY_BEGIN
    base_model = load_model()
    base_model = set_nontrainable_layers(base_model)
    flatten_layer = layers.Flatten()
    dense_layer = layers.Dense(500, activation='relu')
    prediction_layer = layers.Dense(21, activation='sigmoid')

    model = models.Sequential([
        base_model,
        flatten_layer,
        dense_layer,
        prediction_layer
    ])
    # $CHALLENGIFY_END
    return model

model = add_last_layers(model)

In [55]:
from tensorflow.keras import optimizers

def build_model():
    # $CHALLENGIFY_BEGIN
    model = load_model()
    model = add_last_layers(model)

    opt = optimizers.Adam(learning_rate=1e-4)
    model.compile(loss='binary_crossentropy',
                  optimizer=opt,
                  metrics=['categorical_accuracy'])
    return model
    # $CHALLENGIFY_END

model = build_model()

In [56]:
from tensorflow.keras.applications.vgg16 import preprocess_input

X_train = preprocess_input(X_train)
X_val = preprocess_input(X_val)
X_test = preprocess_input(X_test)

In [57]:
from tensorflow.keras.callbacks import EarlyStopping

model = build_model()

es = EarlyStopping(monitor = 'categorical_accuracy',
                   mode = 'max',
                   patience = 5,
                   verbose = 1,
                   restore_best_weights = True)

history = model.fit(X_train, y_train,
                    validation_data=(X_val, y_val),
                    epochs=50,
                    batch_size=16,
                    callbacks=[es])

Epoch 1/50
198/198 [==============================] - 25s 121ms/step - loss: 0.9442 - categorical_accuracy: 0.1956 - val_loss: 0.4528 - val_categorical_accuracy: 0.2081
Epoch 2/50
198/198 [==============================] - 19s 97ms/step - loss: 0.2734 - categorical_accuracy: 0.3912 - val_loss: 0.4988 - val_categorical_accuracy: 0.2766
Epoch 3/50
198/198 [==============================] - 19s 94ms/step - loss: 0.1776 - categorical_accuracy: 0.4255 - val_loss: 0.5227 - val_categorical_accuracy: 0.2589
Epoch 4/50
198/198 [==============================] - 19s 95ms/step - loss: 0.1217 - categorical_accuracy: 0.4420 - val_loss: 0.5996 - val_categorical_accuracy: 0.2817
Epoch 5/50
198/198 [==============================] - 19s 96ms/step - loss: 0.0908 - categorical_accuracy: 0.4347 - val_loss: 0.6186 - val_categorical_accuracy: 0.2538
Epoch 6/50
198/198 [==============================] - 19s 97ms/step - loss: 0.0706 - categorical_accuracy: 0.4353 - val_loss: 0.6622 - val_categorical_accuracy

In [58]:
res_vgg16 = model.evaluate(X_test, y_test)


13/13 [==============================] - 4s 350ms/step - loss: 0.7078 - categorical_accuracy: 0.2582


In [59]:
import numpy as np
from sklearn.metrics import hamming_loss, accuracy_score

y_pred = model.predict(X_test)

y_pred=np.argmax(y_pred, axis=1)
y_true=np.argmax(y_test, axis = 1)

print("accuracy_score:", accuracy_score(y_true, y_pred))
print("Hamming_loss:", hamming_loss(y_true, y_pred))

13/13 [==============================] - 2s 171ms/step
accuracy_score: 0.2582278481012658
Hamming_loss: 0.7417721518987341


In [61]:
alpha_evaluation_score(y_true, y_pred)

0.8083989551601324

In [39]:
y_true

31

In [63]:
np.save('predictions.npy', y_pred)

In [32]:
predictions = np.load('predictions.npy')

FileNotFoundError: [Errno 2] No such file or directory: 'predictions.npy'

In [ ]:
multilabel_binarizer.inverse_transform(y_pred)[0:10]

## Stacking Multimodal Model

In [ ]:
vocab_size

1582

In [ ]:
multilabel_binarizer.classes_.shape

In [ ]:
# define two sets of inputs
inputA = layers.Input(shape = (350,350,3))
inputB = layers.Input(shape=(352,352,3))

# the first branch operates on the first input
x = layers.Embedding(
    input_dim = vocab_size +1,
    input_length = 292,
    output_dim = 100,
    mask_zero = True)
x = layers.Dense(1024, activation = 'relu')(inputA)
x = layers.Dense(21, activation='softmax')(x)
x = models.Model(inputs = inputA, outputs=x)

# the second branch operates on the second input
y = layers.Conv2D(16, kernel_size = (3,3), activation="relu")(inputB)
y = layers.Dense(32, activation="relu")(y)
y = layers.Dense(21, activation="softmax")(y)
y = models.Model(inputs = inputB, outputs=y)

# combine the output of the two branches
combined = layers.concatenate([x.output, y.output])

# apply a FC layer and then a regression prediction on the
# combined outputs

z = layers.Dense(10, activation="relu")(combined)
z = layers.Dense(21, activation="softmax")(z)

# our model will accept the inputs of the two branches and
# then output a single value
model = models.Model(inputs=[x.input, y.input], outputs=z)

## BERT MODEL

In [ ]:
tk = Tokenizer()
tk.fit_on_texts(X_train['plot'])
vocab_size = len(tk.word_index)
print(f'There are {vocab_size} different words in your corpus')
X_token = tk.texts_to_sequences(X_train['plot'])

X_pad = pad_sequences(X_token, dtype = 'float32', padding = 'post')

There are 1582 different words in your corpus


In [ ]:
input_word_ids = layers.Input(shape=(max_seq_length,), dtype=tensorflow.int32,
                                           name="input_word_ids")
input_mask = layers.Input(shape=(max_seq_length,), dtype=tensorflow.int32,
                                       name="input_mask")
segment_ids = layers.Input(shape=(max_seq_length,), dtype=tensorflow.int32,
                                        name="segment_ids")

seq_out = bert_layer([input_ids, input_masks, input_segments])
out = layers.LSTM(128, name='LSTM')(seq_out)
model_lstm = models.Model([input_ids, input_masks, input_segments], out)

In [ ]:
s = "This is a nice sentence."
stokens = tokenizer.tokenize(s)
stokens = ["[CLS]"] + stokens + ["[SEP]"]

input_ids = get_ids(stokens, tokenizer, max_seq_length)
input_masks = get_masks(stokens, max_seq_length)
input_segments = get_segments(stokens, max_seq_length)

pool_embs, all_embs = model.predict([[input_ids],[input_masks],[input_segments]])

In [63]:
!git status

On branch Jess-image-model
Changes to be committed:
  (use "git restore --staged <file>..." to unstage)
	new file:   Jess_Model_3.ipynb

Changes not staged for commit:
  (use "git add <file>..." to update what will be committed)
  (use "git restore <file>..." to discard changes in working directory)
	modified:   ../.gitignore
	modified:   Basic-Model.ipynb
	modified:   Jess-Model-2.ipynb
	modified:   Jess_Model_3.ipynb
	modified:   webscraping_jess2.ipynb
	modified:   yinghui_model_nlp.ipynb

Untracked files:
  (use "git add <file>..." to include in what will be committed)
	.ipynb_checkpoints/Basic-Model-checkpoint.ipynb
	.ipynb_checkpoints/Image-Classification-checkpoint.ipynb
	.ipynb_checkpoints/Jess-Model-2-checkpoint.ipynb
	.ipynb_checkpoints/Jess_Model_3-checkpoint.ipynb
	.ipynb_checkpoints/howard_raw_data-checkpoint.ipynb
	.ipynb_checkpoints/import-checkpoint
	.ipynb_checkpoints/webscraping_jess2-checkpoint.ipynb
	.ipynb_checkpoints/yinghui_model_nlp-checkpoint.ipynb
	C:/
	Downlo

In [3]:
!git add Jess_Model_3.ipynb

In [4]:
!git push origin Jess-image-model

Everything up-to-date
